1. pip install opencv-python insightface onnxruntime
2. manual download of model -> https://github.com/deepinsight/insightface/issues/2335

In [ ]:
import cv2
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import os

In [ ]:
# Initialize the FaceAnalysis module
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(640, 640))

In [ ]:
# load the common face to swap with
common_face_img = cv2.imread('pocketfm/character-profile-image/elena_martin.jpeg')
common_faces = app.get(common_face_img)
if len(common_faces) == 0:
    print("n face detected in the common face image.")
    exit()
common_face = common_faces[0]

# initialize the face swapper
model_path = 'inswapper_128.onnx'
if not os.path.exists(model_path):
    model_path = os.path.expanduser('model/inswapper_128.onnx')
swapper = insightface.model_zoo.get_model(model_path)

### DRIVER BLOCK

In [ ]:
# open the input video
input_video = cv2.VideoCapture('pocketfm/video-in/input_luma_person_2.mp4')

# get video properties
fps = int(input_video.get(cv2.CAP_PROP_FPS))
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# create output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('pocketfm/video-out/luma_output_video.mp4', fourcc, fps, (width, height))

while True:
    ret, frame = input_video.read()
    if not ret:
        break
    # detect faces in the frame
    faces = app.get(frame)
    if len(faces) > 0:
        # swap face for each detected face
        for face in faces:
            frame = swapper.get(frame, face, common_face, paste_back=True)
    # write the frame to the output video
    output_video.write(frame)

# release video objects
input_video.release()
output_video.release()